# Data Cleaning and Preparation

In [31]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

A lot of these methods for cleaning & prepping data can be done with python code, but Pandas has efficient, built-in methods for accomplishing these tasks as well.

## Handling Missing Data

Missing data from a column of floats is specified as NaN by default.

In [4]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

pg 192 has more methods for identifying and dealing with missing data.

-dropna
-fillna
-isnull
-notnull

### Filtering Out Missing Data

In [7]:
# while we can use .isnull and boolean indexing for dropping
# rows w/ nulls, we can also;

from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

# to drop rows will null values, we can use .dropna()
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data
cleaned

# dropna works well with dropping rows w/ null values..
# but it can obliterate larage chunks of our database very easily

,0,1,2
0,1.0,6.5,3.0


In [10]:
# maybe we want to keep rows with some values.
# we can specifically drop rows where all values are null;

data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
# what if we have a column with nulls or that is full of nulls?
# axis=1 works on columns

data[4] = NA
data
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
# it's not all or nothing though, we can set a threshold
# the below will keep any rows with at least 2 non-null values

df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data

We dont' always want to remove nulls. We can also choose to fill each null with a specified value (mean, mode, or some other value depending on the application/data).

In [14]:
# .fillna(argument). the below changes nulls to 0;
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [15]:
# we can specify the fill value by column, user dict-like syntax

df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [16]:
# because fillna returns an object, we can either assign it to
# something new, or set inplace=True to update the original.

_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [17]:
# itnerpolation methods available for reindexing also work for
# fillna. below we make index 2 down at index 1 null, and do the same
# index 4 at index 2 to the end.

# this adds the last value in a column to the rows below it

df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [18]:
# we can pass in anything that follows the same datatype
# maybe we want to keep our mean the same, so pass in the mean
# of the values we have to fillna

data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

pg. 197 has fillna function arguments

## Data Transformation

When working with data, there are any number of reasons there may be duplicate rows.

### Removing Duplicates

In [19]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [20]:
# let's check for duplicate entries

data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [21]:
# in the words of words of everyone's grandma in New Jersey,
# "get the heck outta heeeeeeere"

#.drop_duplicates() will remove duplicate entries
# this returns a dataframe, so keeps the original intact
# unless inplace=True.

data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [22]:
data['v1'] = range(7)
# we can specify a column to drop duplicates from

data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [ ]:
# or drop duplicates from multiple columns.
# by default the first occurrence will be kept
# keep='last' keeps the last time that value is found

data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping

for many datasets you may want to transform data based on values in an array.

let's look at types of meat.

In [24]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [26]:
# we have the types of meat, but what if we wanted to add 
# the type of animal the meat comes from?
# we can set up a dict matching current values to what we 
# want to add.

meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [27]:
# lowercased to avoid mismatches based on casing
# then we map that data

lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [28]:
# we can do the same thing with a lambda.
# 'x' being each value in the food column

data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

Filling in missing data with fillna is a special case of more general value replacement. Map can be used to modify a subset of values, but replace offers a simpler & more flexible way of doing this.

In [29]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [30]:
# for this data, let's asssume -999 is the sentinel value
# for missing values.

data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [32]:
# maybe -999 AND -1000 are known sentinel values for None
# replacing values in multiple columns

data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [33]:
# doing it this way, -999 becomes NaN
# -1000 becomes 0

data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [34]:
# the above arguments can also be passed as a dict

data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [35]:
# like values, we can also change the names of our axis labels
# the process is similar to changing row values

In [36]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [37]:
# makes 4 letter abbreviations [:4] returned as uppercase.
# df.index is how we set our index values, and since we have 
# a list we want to use, we can use .map in conjunction

transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [38]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [40]:
# the rename function also works

data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [41]:
# rename can also be used in conjuction with a dict-like
# object, providing new values for a subset of the axis labels

data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [ ]:
# as always, inplace keeps us from having to
# save the change to a different df before applying
# to our working df.

data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

We can create bins for ranges of values if we want to examine data between certain bounds.

Age is a good example of this.

In [42]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [43]:
# to make bins based on our range values, we use pd.cut.


bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [44]:
# what was returned is called a 'Categorical' object.
# we now have categories based on our bins.

# .codes shows us which bin each age value falls in to
# .categories shows the groupings we've specified
# .value_counts(cats) show how many rows are in each category

cats.codes
cats.categories
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [45]:
# Categories can also take strings as their names.

group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [46]:
# bins don't have to be ranges of values,
# they can also be set to split data in to chunks

# take another look at this

data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64, right]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

In [48]:
# qcut is similar to cut, but splits according to quantiles.
# the below splits in to 4 bins, 25%ea quantile.
# because the data may not be evenly distrubuted 
# by quantiles, our bins are likely to be different sizes.

data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

(-3.746, -0.612]    250
(-0.612, -0.011]    250
(-0.011, 0.681]     250
(0.681, 3.26]       250
dtype: int64

In [49]:
# in addition to binning by a specified number of bins,
# we can also set quantiles to exam
# below has a 0-10% bin, 10-50% bin, 50-90% bin, & 90-100% bin.

pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.299, -0.011], (-0.011, 1.338], (-1.299, -0.011], (-0.011, 1.338], (-0.011, 1.338], ..., (-3.746, -1.299], (-0.011, 1.338], (-0.011, 1.338], (-0.011, 1.338], (-1.299, -0.011]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.746, -1.299] < (-1.299, -0.011] < (-0.011, 1.338] < (1.338, 3.26]]

### Detecting and Filtering Outliers

In [50]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.016402,0.042815,-0.037415,-0.031117
std,1.015199,0.989036,0.963336,0.988520
min,-3.645860,-3.333767,-2.901831,-3.428254
25%,-0.664374,-0.612162,-0.704993,-0.729106
50%,0.038757,0.028040,-0.062375,-0.064342
75%,0.722198,0.695298,0.664260,0.630153
max,2.763474,3.525865,2.611678,3.366626


In [51]:
# we've got our normally distrubted table of random #s.
# let's find values in one of the columns that 
# has an absolute value above 3.


col = data[2]
col[np.abs(col) > 3]

Series([], Name: 2, dtype: float64)

In [ ]:
# to select all rows above absolute val 3,
# we can use a boolean dataframe.
data[(np.abs(data) > 3).any(1)]

In [52]:
# this is code to make the highest value 3. if it
# falls out of bounds, we set it equal to 3 * its original
# sign.

data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.017615,0.042807,-0.037415,-0.030947
std,1.011227,0.985647,0.963336,0.985609
min,-3.000000,-3.000000,-2.901831,-3.000000
25%,-0.664374,-0.612162,-0.704993,-0.729106
50%,0.038757,0.028040,-0.062375,-0.064342
75%,0.722198,0.695298,0.664260,0.630153
max,2.763474,3.000000,2.611678,3.000000


In [54]:
# sign is shown as 1.0 or -1.0. Easy multiplication
# when using .sign .
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,1.0,-1.0
4,-1.0,1.0,-1.0,-1.0


### Permutation and Random Sampling

In [59]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

# for taking a random sample from our DataFrame

array([3, 1, 4, 2, 0])

In [60]:
df
df.take(sampler)

# .take takes random ilocs, take a look at the returned rows

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3


In [61]:
# to take a random subset without replacing values,
# .sample, and how n=how-many-samples-to-take;

df.sample(n=3)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
0,0,1,2,3


In [62]:
# to generate a sample with replacement (to allow repeat
# choices), pass replace=True;

choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

1    7
1    7
0    5
3    6
4    4
0    5
1    7
2   -1
2   -1
1    7
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorial variable into a 'dummy' or 'indicator' matrix. 

If DataFrame has k distinct values, you would dreive a matrix or DataFrame with k columns containing all 1s and 0s.

Pandas has a get_dummies function, but doing this ourselves isn't difficult.

In [63]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [65]:
# when altering data with dummies, we can prefix
# our column names.

# this way we can join them later using .join

dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [66]:
#

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

/var/folders/6k/dfzdrspj2yn7xlvg1wz81x0m0000gp/T/ipykernel_62303/1263338300.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [69]:
# some of those genres are split by pipes |. So
# our unique count isn't exactly accurate.

# adding indicators to our genres will take some wrangling.

# wrangling ensues

all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [71]:
genres

# this gives us our comprehensive list of distinct genres.

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [72]:
# one way to construct the indicator is to start w/ 
# a df of all zeros.

zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [73]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [ ]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [74]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             0.0
Genre_Children's                            0.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

In [75]:
# after this series of steps, we can see the distinct
# categories each movies falls in to by seeing the bins
# they appear in

# ** get more familiar with this. looks like a really
# useful wrangling strategy.

np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

### String Object Methods

Most string manupilation can be done with str object methods. Pandas has some added capabilities.

In [76]:
# well this isn't good for a list of columns..

val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [77]:
# when we want to remove whitespace

pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [78]:
# is your string joining too cumbersome?

first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [79]:
# join strings with one line! using .join !
# supplies unlimited

'::'.join(pieces)

'a::b::guido'

In [80]:
# did you remember the string, honey?

'guido' in val
# True
val.index(',')
# 1
val.find(':')

-1

In [81]:
val.index(':') #!! remember the difference between find
# and .index. .f

ValueError: substring not found

In [82]:
val.count(',')

2

In [83]:
val.replace(',', '::')
val.replace(',', '')

'ab  guido'

### Regular Expressions

regex in python is used via the re module

In [1]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)
#split the string by empty space

['foo', 'bar', 'baz', 'qux']

In [2]:
# we can save a regex expression to a variable by using re.compile. This way we can use it as a regex object w/ 
# those specified parameters already passed in.
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [4]:
# to find all substrings matching the regex, use .findall()
regex.findall(text)

['    ', '\t ', '  \t']

In [7]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
# finds [alphanumeric  & symbols] followed by an @ then [alphanumeric] then . then [A-Z], ignores case
#finding an email address in a string
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [11]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [14]:
# .search() returns the start & end point for the first match 

m = regex.search(text)
m
# text[m.start():m.end()]

<re.Match object; span=(5, 20), match='dave@google.com'>

In [16]:
# regex.search returns an object with start & end properties that are ints for index values.

text[m.start():m.end()]

'dave@google.com'

In [13]:
print(regex.match(text))

None


In [18]:
# replaces our regex pattern with the given word... in this case e-mail addresses become 'REDACTED'

print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [26]:
# by putting parentheses around our regex pieces, we can split the results in to groups. For the below we
# can see a username, domain name, and domain suffix.

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [27]:
# regex_object.groups() will show the groups, each group being its own tuple.
# this way we can split up an email address

m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [28]:
# well would ya look at that. this is practically ready for a dataframe.

regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [29]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### pg. 216 has more Regular Expression methods

### Vectorized String Functions in pandas

In [43]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [33]:
# our .str.... methods are good 
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [34]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [41]:
# these same string methods can be used in conjunction with our regular expressions

matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [44]:
matches.str.get(1)
# matches.str[0]

AttributeError: Can only use .str accessor with string values!

In [45]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [46]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion